<a href="https://colab.research.google.com/github/mathjams/machine-learning-basics/blob/main/supervised_learning_linear_and_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#creating dataset
x_train, y_train = load_data()

**Linear Regression**

For linear regression, $f_{w,b}=wx+b$ is the function.

Want to find the best fit with the smallest cost $$J(w,b)=\frac{1}{2m}\sum_{i=0}^{m-1}(f_{w,b}(x^{(i)})-y^{(i)})^2.$$

The following code calculates $J(w,b)$ for a specific training set $x,y$ and parameters $w$ and $b$.

In [ ]:
# Create a scatter plot of the data. To change the markers to red "x",
# we used the 'marker' and 'c' parameters
plt.scatter(x_train, y_train, marker='x', c='r')

# Set the title
plt.title("Profits vs. Population per city")
# Set the y-axis label
plt.ylabel('Profit in $10,000')
# Set the x-axis label
plt.xlabel('Population of City in 10,000s')
plt.show()

In [ ]:
def compute_cost(x, y, w, b):
    """
    Computes the cost function for linear regression.

    Args:
        x (ndarray): Shape (m,) Input to the model (ex Population of cities)
        y (ndarray): Shape (m,) Output (ex Profit)
        w, b (scalar): Parameters of the model

    Returns
        total_cost (float): The cost of using w,b as the parameters for linear regression
               to fit the data points in x and y
    """
    m = x.shape[0]
    total_cost = 0
    for i in range(m):
        prediction=w*x[i]+b
        cost=(prediction-y[i])**2
        total_cost+=cost/(2*m)

    return total_cost

Gradient Descent slowly changes the parameters $w,b$ until convergence. In particular, they adjust so that $w,b$ approach the local minimum. In particular, $b$ adjusts to
$b -  \alpha \frac{\partial J(w,b)}{\partial b}$ and $w$ adjusts to $w -  \alpha \frac{\partial J(w,b)}{\partial w}$.

We can calculate $\frac{\partial J(w,b)}{\partial b}=\frac{1}{m}\sum_{i=0}^{m-1} (f_{w,b}(x^{(i)})-y^{(i)})$ and $\frac{\partial J(w,b)}{\partial w}=\frac{1}{m}\sum_{i=0}^{m-1} (f_{w,b}(x^{(i)})-y^{(i)})(x^{(i)})$.

The code below calculates the gradients $\frac{\partial J(w,b)}{\partial b}$ and $\frac{\partial J(w,b)}{\partial w}$ explicitly.


In [ ]:
# GRADED FUNCTION: compute_gradient
def compute_gradient(x, y, w, b):
    """
    Computes the gradient for linear regression
    Args:
      x (ndarray): Shape (m,) Input to the model (Population of cities)
      y (ndarray): Shape (m,) Label (Actual profits for the cities)
      w, b (scalar): Parameters of the model
    Returns
      dj_dw (scalar): The gradient of the cost w.r.t. the parameters w
      dj_db (scalar): The gradient of the cost w.r.t. the parameter b
     """
    m = x.shape[0]
    dj_dw = 0
    dj_db = 0
    for i in range(m):
        fwb=w*x[i]+b
        dj_dw+=(fwb-y[i])*x[i]/m
        dj_db+=(fwb-y[i])/m
    return dj_dw, dj_db

In [ ]:
def gradient_descent(x, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters):
    """
    Performs batch gradient descent to learn theta. Updates theta by taking
    num_iters gradient steps with learning rate alpha

    Args:
      x :    (ndarray): Shape (m,)
      y :    (ndarray): Shape (m,)
      w_in, b_in : (scalar) Initial values of parameters of the model
      cost_function: function to compute cost
      gradient_function: function to compute the gradient
      alpha : (float) Learning rate
      num_iters : (int) number of iterations to run gradient descent
    Returns
      w : (ndarray): Shape (1,) Updated values of parameters of the model after
          running gradient descent
      b : (scalar)                Updated value of parameter of the model after
          running gradient descent
    """
    m = len(x)

    # An array to store cost J and w's at each iteration — primarily for graphing later
    J_history = []
    w_history = []
    w = copy.deepcopy(w_in)  #avoid modifying global w within function
    b = b_in

    for i in range(num_iters):

        # Calculate the gradient and update the parameters
        dj_dw, dj_db = gradient_function(x, y, w, b )

        # Update Parameters using w, b, alpha and gradient
        w = w - alpha * dj_dw
        b = b - alpha * dj_db

        # Save cost J at each iteration
        if i<100000:      # prevent resource exhaustion
            cost =  cost_function(x, y, w, b)
            J_history.append(cost)

        # Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iters/10) == 0:
            w_history.append(w)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")

    return w, b, J_history, w_history #return w and J,w history for graphing

**Logistic Regression**

For Logistic Regression, we use the Sigmoid function $$f_{w,b}(x)=g(w\cdot x+b)=\frac{1}{1+e^{-(w\cdot x+b)}}$$.

Below we provide a code for the Sigmoid function, which returns all the components of $z$ under $g(z)$.

In [ ]:
def sigmoid(z):
    """
    Compute the sigmoid of z

    Args:
        z (ndarray): A scalar, numpy array of any size.

    Returns:
        g (ndarray): sigmoid(z), with the same shape as z

    """
    try:
        g=np.zeros(z.shape)
        if len(z.shape)>1:
            for i in range(z.shape[0]):
                for j in range(z.shape[1]):
                    g[i][j]=1/(1+math.exp(-z[i][j]))
        else:
            for i in range(z.shape[0]):
                g[i]=1/(1+math.exp(-z[i]))
        return g
    except:
        return 1/(1+math.exp(-z))

The loss function for a sigmoid function is a bit different, where (note that $w$ is vector with parameters of many different features) $$J(w,b)=\frac{1}{m}\sum_{i=0}^{m-1} loss(f_{w,b}(x^{(i)}), y^{(i)})$$ with a differently defined $loss$ function than for linear regression. In this case, $y^{(i)}$ is either $1$ or $0$, discrete values instead of continuous labels. The loss function is defined as $$loss(f_{w,b}(x^{(i)}), y^{(i)})=-y^{(i)}\log (f_{w,b}(x^{(i)}))-(1-y^{(i)})\log (1-f_{w,b}(x^{(i)}))$$ where $f_{w,b}(x^{(i)})=g(w\cdot x^{(i)}+b)$ is the model's prediction.

Below computes the cost for training $X$ and outputs $y$ and parameters $w,b$.

In [ ]:
def compute_cost(X, y, w, b, *argv):
    """
    Computes the cost over all examples
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value
      w : (ndarray Shape (n,))  values of parameters of the model
      b : (scalar)              value of bias parameter of the model
      *argv : unused, for compatibility with regularized version below
    Returns:
      total_cost : (scalar) cost
    """
    m, n = X.shape

    total_cost=0
    for i in range(m):
        sum=b
        for j in range(n):
            sum+=X[i][j]*w[j]
        fwb=sigmoid(sum)
        total_cost+=(-y[i]*math.log(fwb)-(1-y[i])*math.log(1-fwb))/m

    return total_cost

The gradient function for logistic and linear functions are the same, and so is gradient descent.

In [ ]:
def compute_gradient(X, y, w, b, *argv):
    """
    Computes the gradient for logistic regression

    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value
      w : (ndarray Shape (n,))  values of parameters of the model
      b : (scalar)              value of bias parameter of the model
      *argv : unused, for compatibility with regularized version below
    Returns
      dj_dw : (ndarray Shape (n,)) The gradient of the cost w.r.t. the parameters w.
      dj_db : (scalar)             The gradient of the cost w.r.t. the parameter b.
    """
    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.
    for i in range(m):
        z_wb = 0
        for j in range(n):
            z_wb += X[i][j]*w[j]
        z_wb += b
        f_wb = sigmoid(z_wb)

        dj_db += (f_wb-y[i])/m

        for j in range(n):
            dj_dw[j] = dj_dw[j]+(f_wb-y[i])*X[i][j]/m
    return dj_db, dj_dw

Now, given a dataset you can use this learned model on a dataset to predict whether it is $0$ or $1$ and select the threshold for doing so.

In [ ]:
def predict(X, w, b):
    """
    Predict whether the label is 0 or 1 using learned logistic
    regression parameters w

    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      w : (ndarray Shape (n,))  values of parameters of the model
      b : (scalar)              value of bias parameter of the model

    Returns:
      p : (ndarray (m,)) The predictions for X using a threshold at 0.5
    """
    # number of training examples
    m, n = X.shape
    p = np.zeros(m)
    # Loop over each example
    for i in range(m):
        z_wb = 0
        # Loop over each feature
        for j in range(n):
            # Add the corresponding term to z_wb
            z_wb += X[i][j]*w[j]

        # Add bias term
        z_wb += b

        # Calculate the prediction for this example
        f_wb = sigmoid(z_wb)

        # Apply the threshold
        if f_wb<0.5:
            p[i] = 0
        else:
            p[i]=1
    return p

**Regularization**

To avoid overfitting, we add a regularization term $\frac{\lambda}{2m}\sum_{i=0}^{n-1} w_i^2$ to $J(w,b)$ to decrease the coefficients of features that aren't as important (kind of like pinalize them). For example, we might assign features like $x_i^2$ to make a nonlinear function, but this might not be the best fit (maybe a linear function is the best fit!) and even cause overfitting which makes it harder to generalize.

The code below adds regularization.

In [ ]:
def compute_cost_reg(X, y, w, b, lambda_ = 1):
    """
    Computes the cost over all examples
    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value
      w : (ndarray Shape (n,))  values of parameters of the model
      b : (scalar)              value of bias parameter of the model
      lambda_ : (scalar, float) Controls amount of regularization
    Returns:
      total_cost : (scalar)     cost
    """

    m, n = X.shape
    # Calls the compute_cost function that you implemented above
    cost_without_reg = compute_cost(X, y, w, b)

    # You need to calculate this value
    reg_cost = 0.

    for i in range(n):
        reg_cost+=lambda_*w[i]**2/(2*m)

    # Add the regularization cost to get the total cost
    total_cost = cost_without_reg + reg_cost

    return total_cost

The regularization term changes $\frac{\partial J(w,b)}{\partial w_j}$ by a term of $\frac{\lambda}{m}w_j$.  


In [ ]:
def compute_gradient_reg(X, y, w, b, lambda_ = 1):
    """
    Computes the gradient for logistic regression with regularization

    Args:
      X : (ndarray Shape (m,n)) data, m examples by n features
      y : (ndarray Shape (m,))  target value
      w : (ndarray Shape (n,))  values of parameters of the model
      b : (scalar)              value of bias parameter of the model
      lambda_ : (scalar,float)  regularization constant
    Returns
      dj_db : (scalar)             The gradient of the cost w.r.t. the parameter b.
      dj_dw : (ndarray Shape (n,)) The gradient of the cost w.r.t. the parameters w.

    """
    m, n = X.shape

    dj_db, dj_dw = compute_gradient(X, y, w, b)

    for i in range(n):
        dj_dw[i]+=lambda_*w[i]/m

    return dj_db, dj_dw